In [21]:
from datetime import datetime
from pathlib import Path

import datefinder
import pandas as pd
import tqdm.notebook

In [22]:
data_dir = Path('data/.')
data_list = [f for f in data_dir.glob('*.csv')]
len(data_list)

1025

In [150]:
def parse_date(data_path) -> str:
    date_time = [t for t in datefinder.find_dates(str(data_path))]
    date_str = datetime.strftime(date_time[0], '%Y.%m.%d')
    return date_str

In [151]:
def load_and_process_data_from_path(data_path):
    df = pd.read_csv(data_path)
    if 'Country_Region' not in df.columns:
        df.rename(columns={'Country/Region': 'Country_Region'}, inplace=True)
    df.loc[df['Country_Region'] == 'Mainland China', 'Country_Region'] = 'China'
    df.loc[df['Country_Region'] == 'Korea, South', 'Country_Region'] = 'South Korea'
    df.loc[df['Country_Region'] == 'Taiwan*', 'Country_Region'] = 'Taiwan'
    return df

def extract_confirmed(data_path):
    date_str = parse_date(data_path)
    df = load_and_process_data_from_path(data_path)
    df = (df
        .loc[:, ['Country_Region', 'Confirmed']]
        .rename(columns={'Confirmed': date_str})
        .groupby('Country_Region', as_index=False).sum(date_str)
    )
    return df

def extract_lat_long(data_path):
    df = load_and_process_data_from_path(data_path)
    df = df.loc[:, ['Country_Region', 'Lat', 'Long_']]
    return df

In [152]:
lat_long = extract_lat_long(data_list[-1])
lat_long.drop_duplicates('Country_Region', inplace=True) # give up some province's lat_long
lat_long

,Country_Region,Lat,Long_
0,Afghanistan,33.93911,67.709953
1,Albania,41.15330,20.168300
2,Algeria,28.03390,1.659600
3,Andorra,42.50630,1.521800
4,Angola,-11.20270,17.873900
...,...,...,...
4004,Winter Olympics 2022,39.90420,116.407400
4005,Antarctica,-71.94990,23.347000
4008,"Korea, North",40.33990,127.510100
4010,Nauru,-0.52280,166.931500


In [153]:
df_all = extract_confirmed(data_list[0])
for filepath in tqdm.tqdm(data_list[1:]):
    df = extract_confirmed(filepath)
    df_all = pd.merge(df_all, df, on='Country_Region', how='outer')

100%|██████████| 1024/1024 [01:24<00:00, 12.17it/s]


In [154]:
df_all_fillna = df_all.fillna(0)
df_all_fillna.head()

,Country_Region,2022.01.21,2020.02.26,2022.01.20,2020.02.27,2021.07.04,2021.07.05,2022.02.12,2022.02.13,2021.12.31,...,2021.10.12,2021.07.29,2021.09.30,2021.07.28,2022.03.03,2022.03.02,2021.05.26,2021.05.27,2021.06.15,2021.06.14
0,Afghanistan,159516.0,5.0,159303.0,5.0,124748.0,125937.0,170152.0,170604.0,158084.0,...,155599.0,145996.0,155174.0,145552.0,174214.0,174073.0,67743.0,68366.0,93272.0,91458.0
1,Albania,244182.0,0.0,241512.0,0.0,132535.0,132537.0,267551.0,268008.0,210224.0,...,175664.0,132999.0,170131.0,132952.0,271825.0,271825.0,132244.0,132264.0,132469.0,132461.0
2,Algeria,232325.0,1.0,230470.0,1.0,141471.0,141966.0,261226.0,261752.0,218432.0,...,204790.0,168668.0,203359.0,167131.0,265130.0,265079.0,127646.0,127926.0,134115.0,133742.0
3,Andorra,33025.0,0.0,32201.0,0.0,13918.0,13918.0,37140.0,37140.0,23740.0,...,15307.0,14655.0,15222.0,14586.0,38342.0,38249.0,13671.0,13682.0,13828.0,13826.0
4,Angola,95676.0,0.0,95220.0,0.0,39230.0,39300.0,98514.0,98514.0,81593.0,...,61794.0,42486.0,56583.0,42288.0,98746.0,98746.0,33338.0,33607.0,36921.0,36790.0


In [156]:
df_all_sorted = df_all_fillna.reindex(sorted(df_all_fillna.columns[1:]), axis=1)
df_all_sorted.insert(loc=0, column='Country_Region', value=df_all_fillna['Country_Region'])
df_all_sorted = df_all_sorted.sort_values('2022.11.11', ascending=False)
df_all_sorted

,Country_Region,2020.01.22,2020.01.23,2020.01.24,2020.01.25,2020.01.26,2020.01.27,2020.01.28,2020.01.29,2020.01.30,...,2022.11.02,2022.11.03,2022.11.04,2022.11.05,2022.11.06,2022.11.07,2022.11.08,2022.11.09,2022.11.10,2022.11.11
186,US,1.0,1.0,2.0,2.0,5.0,5.0,5.0,5.0,5.0,...,97627774.0,97698174.0,97736547.0,97741608.0,97749152.0,97787763.0,97817525.0,97913411.0,97978279.0,97990681.0
80,India,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,44655926.0,44658365.0,44659447.0,44660579.0,44661504.0,44660293.0,44660293.0,44660293.0,44664810.0,44665643.0
63,France,0.0,0.0,2.0,3.0,3.0,3.0,4.0,5.0,5.0,...,37068741.0,37110800.0,37140238.0,37140238.0,37140238.0,37191901.0,37232493.0,37261364.0,37288432.0,37288432.0
67,Germany,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,4.0,...,35728277.0,35784912.0,35823771.0,35823771.0,35823771.0,35884834.0,35932654.0,35971322.0,36005025.0,36033394.0
24,Brazil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34837035.0,34846308.0,34849063.0,34849063.0,34849063.0,34855492.0,34855492.0,34855492.0,34889576.0,34908198.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,Taipei and environs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
219,Viet Nam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
220,occupied Palestinian territory,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
221,Cruise Ship,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [157]:
filename = 'result'
df_all_sorted.to_csv(f'{filename}.csv')

In [158]:
# Add Lat and Long data
# df_with_lat_long = lat_long.set_index('Country_Region').join(df_all_sorted.set_index('Country_Region'))
# df_with_lat_long.fillna(0, inplace=True)
# df_with_lat_long.to_csv('result_lat_long.csv')
# df_with_lat_long

---
## Retrieve country flags

In [159]:
base_url = 'https://cdn.countryflags.com/thumbs/{}/flag-800.png'

In [160]:
def make_flag_url(country_name):
    country_name = country_name.lower().replace(' ', '-')
    if country_name == "us":
        country_name = 'united-states-of-america'
    flag_url = base_url.format(country_name)
    return(flag_url)

In [161]:
df_with_flags = df_all_sorted.copy()

In [162]:
flag_urls_series = df_with_flags['Country_Region'].apply(make_flag_url)
df_with_flags.insert(1, 'flag_url', flag_urls_series)
df_with_flags.loc[df_with_flags['Country_Region'] == 'Others', 'flag_url'] = '' # Remove others' flag url
df_with_flags

,Country_Region,flag_url,2020.01.22,2020.01.23,2020.01.24,2020.01.25,2020.01.26,2020.01.27,2020.01.28,2020.01.29,...,2022.11.02,2022.11.03,2022.11.04,2022.11.05,2022.11.06,2022.11.07,2022.11.08,2022.11.09,2022.11.10,2022.11.11
186,US,https://cdn.countryflags.com/thumbs/united-sta...,1.0,1.0,2.0,2.0,5.0,5.0,5.0,5.0,...,97627774.0,97698174.0,97736547.0,97741608.0,97749152.0,97787763.0,97817525.0,97913411.0,97978279.0,97990681.0
80,India,https://cdn.countryflags.com/thumbs/india/flag...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,44655926.0,44658365.0,44659447.0,44660579.0,44661504.0,44660293.0,44660293.0,44660293.0,44664810.0,44665643.0
63,France,https://cdn.countryflags.com/thumbs/france/fla...,0.0,0.0,2.0,3.0,3.0,3.0,4.0,5.0,...,37068741.0,37110800.0,37140238.0,37140238.0,37140238.0,37191901.0,37232493.0,37261364.0,37288432.0,37288432.0
67,Germany,https://cdn.countryflags.com/thumbs/germany/fl...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,...,35728277.0,35784912.0,35823771.0,35823771.0,35823771.0,35884834.0,35932654.0,35971322.0,36005025.0,36033394.0
24,Brazil,https://cdn.countryflags.com/thumbs/brazil/fla...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34837035.0,34846308.0,34849063.0,34849063.0,34849063.0,34855492.0,34855492.0,34855492.0,34889576.0,34908198.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,Taipei and environs,https://cdn.countryflags.com/thumbs/taipei-and...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
219,Viet Nam,https://cdn.countryflags.com/thumbs/viet-nam/f...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
220,occupied Palestinian territory,https://cdn.countryflags.com/thumbs/occupied-p...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
221,Cruise Ship,https://cdn.countryflags.com/thumbs/cruise-shi...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [163]:
df_with_flags.loc[df_with_flags['Country_Region'] == 'Others', ['flag_url']]

,flag_url
203,


In [164]:
df_with_flags['flag_url'][186]

'https://cdn.countryflags.com/thumbs/united-states-of-america/flag-800.png'

In [165]:
df_with_flags.to_csv('result_with_flags.csv')

---
Another Way to get CSSE data

In [56]:
import akshare
data_ak = akshare.covid_19_csse_global_confirmed()
data_ak.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/6/22,11/7/22,11/8/22,11/9/22,11/10/22,11/11/22,11/12/22,11/13/22,11/14/22,11/15/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,203681,203829,203942,204094,204287,204392,204417,204510,204610,204724
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,333055,333058,333071,333088,333103,333125,333138,333156,333161,333197
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,270873,270881,270891,270906,270917,270924,270929,270939,270952,270969
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,46588,46588,46588,46664,46664,46664,46664,46664,46664,46664
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,103131,103131,103131,103131,103131,103131,103131,103131,103131,103131
